In [2]:
!pip install tensorflow

In [3]:
%tensorflow_version 2.x

In [4]:
import tensorflow as tf
print(tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.7/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [5]:
string=tf.Variable('string', tf.string)

In [6]:
rank_tensor=tf.Variable([["test","ok"],["test","ok"],["test","ok"]],tf.string)
rank_tensor.shape

TensorShape([3, 2])

type of tensors
1- variable
2-constant
3-placeholder
4-sparse tensor


In [7]:
tf.dtypes.DType

tensorflow.python.framework.dtypes.DType

In [47]:
# with tf.Session() as sess:
#   string.eval()

In [9]:
pip install -q sklearn

In [12]:
from __future__ import absolute_import,division,print_function,unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as feature_column
import tensorflow as tf

In [18]:
#titanic data

dftrain=pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval=pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')

y_train=dftrain.pop('survived')
y_eval=dfeval.pop('survived')

In [19]:
dftrain.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


create feature columns

In [38]:
CATEGORICAL_COLUMNS=['sex','n_siblings_spouses','parch','class','deck','embark_town','alone']
NUMERIC_COLUMNS=['age','fare'] 

In [39]:
feature_columns=[]

for feature_name in CATEGORICAL_COLUMNS:
  vocabulary=dftrain[feature_name].unique() # get unque value of the colmn like (first thired,secon)
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name,vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column( feature_name,dtype=tf.float32))

print(feature_columns)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def

- batch size is to give our data to the model in small batches to make the model fast in training if we have large dataset

- epoch is the number of time the model need to see out complete taining dataset
so we can increase that by the time so the model become better but not too much so we avoid overfitting which is the model see the taining so much but perform bad with testing data

- input function is tensorflow way to to pass the data

In [42]:
def make_input_fn(data_df,label_df,num_epochs=10,shuffle=True,batch_size=32): #shuffle if we will randomise the data not with the order
  def input_function():
    #pass dictionary of variables and y to create dataset type tensor
    ds=tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))
    if shuffle:
      ds=ds.shuffle(1000) #random order of the data
    #split dataset to blocks by batches and repeat epochs time
    ds=ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function
  
tain_input_fn=make_input_fn(dftrain,y_train)
eval_input_fn=make_input_fn(dfeval,y_eval,num_epochs=1,shuffle=False) #becose we dont train here so no need for epochs or shuffle


In [45]:
linear_est=tf.estimator.LinearClassifier(feature_columns=feature_columns) # create classifier with expected input features
linear_est.train(tain_input_fn) #train using the input fnction dataset
result=linear_est.evaluate(eval_input_fn) #get model netrics from testing. data
clear_output() #clear consoke output
print(result['accuracy']) #result variable is dicrionart od states of the model

0.7689394


In [46]:
print(result)

{'accuracy': 0.7689394, 'accuracy_baseline': 0.625, 'auc': 0.8385675, 'auc_precision_recall': 0.7856275, 'average_loss': 0.47437903, 'label/mean': 0.375, 'loss': 0.4656272, 'precision': 0.70212764, 'prediction/mean': 0.35029987, 'recall': 0.6666667, 'global_step': 200}
